In [1]:
import pandas as pd

In [2]:
df_train=pd.read_csv('modified_data_train.csv')
df_test=pd.read_csv('modified_data_test.csv')

In [3]:
df_train.head()

,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmeas_38,xmeas_39,xmeas_40,xmeas_41,xmv_1,xmv_2,xmv_3,xmv_4,xmv_5,xmv_10
0,0.0,2.0,21,0.26775,3630.7,4512.7,9.2949,26.917,42.466,2701.7,...,0.84308,0.089922,52.845,43.879,62.838,54.111,26.634,60.788,22.682,40.947
1,0.0,2.0,22,0.26725,3645.0,4459.4,9.2806,26.676,42.548,2704.6,...,0.84308,0.089922,52.845,43.879,62.592,54.749,26.683,62.394,22.758,40.933
2,0.0,2.0,23,0.25875,3712.7,4503.8,9.4355,26.791,42.003,2708.2,...,0.84308,0.089922,52.845,43.879,63.004,54.295,24.993,59.951,22.822,41.592
3,0.0,2.0,24,0.25368,3690.8,4512.4,9.3769,26.847,42.168,2709.4,...,0.84308,0.089922,52.845,43.879,63.381,54.193,25.491,62.203,22.790,41.633
4,0.0,2.0,25,0.24452,3637.5,4494.1,9.2725,26.962,42.451,2706.5,...,0.84308,0.089922,52.845,43.879,62.087,53.604,24.069,62.409,22.490,41.209


In [4]:
df_test.head()

,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmeas_40,xmeas_41,xmv_1,xmv_3,xmv_4,xmv_5,xmv_6,xmv_9,xmv_10,xmv_11
0,0,2.0,21,0.25673,3668.3,4510.3,9.3270,26.645,42.457,2702.8,...,53.435,43.598,62.260,25.195,62.105,21.872,40.307,46.697,40.220,17.833
1,0,2.0,22,0.25728,3644.0,4482.8,9.3429,26.699,42.166,2704.1,...,53.435,43.598,62.680,25.141,63.101,22.188,40.410,46.728,41.331,17.777
2,0,2.0,23,0.30579,3692.9,4515.2,9.2993,26.911,42.110,2704.3,...,53.435,43.598,63.068,30.283,61.538,22.238,40.996,46.742,40.438,17.927
3,0,2.0,24,0.30538,3729.0,4512.3,9.3040,26.702,42.511,2704.7,...,53.435,43.598,62.446,30.324,61.499,22.183,40.839,46.883,41.357,20.300
4,0,2.0,25,0.27300,3678.1,4517.3,9.2570,26.836,42.018,2707.7,...,53.435,43.598,62.302,26.451,63.040,22.605,38.371,46.922,40.912,16.061


First we are making model to check whether there is any fault or not

In [5]:
df_train['binary_fault'] = df_train['faultNumber'].apply(lambda x: 0 if x == 0 else 1)
df_test['binary_fault'] = df_test['faultNumber'].apply(lambda x: 0 if x == 0 else 1)

The column 'binary fault' is our target. And the columns faultNumber, SimmulationRun and sample do not affect it. Therefore, we are dropping those columns in our training data.

In [6]:
drop_cols = ['faultNumber', 'simulationRun', 'sample', 'binary_fault']
X_train = df_train.drop(columns=drop_cols, errors='ignore')
y_train = df_train['binary_fault']

In [7]:
X_test =  df_test.drop(columns=drop_cols, errors='ignore')
y_test =  df_test['binary_fault']

In [8]:
common_cols = sorted(list(set(X_train.columns).intersection(set(X_test.columns))))
X_train = X_train[common_cols]
X_test  = X_test[common_cols]

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.2,random_state=42)

In [10]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()

In [11]:
rf.fit(X_train,y_train)

RandomForestClassifier()

In [12]:
y_val1=rf.predict(X_val)

In [13]:
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_val, y_val1)
print('Vakidation Accuracy:', accuracy)
print('Classification Report:')
print(classification_report(y_val, y_val1))

Vakidation Accuracy: 0.9400077160493827
Classification Report:
              precision    recall  f1-score   support

           0       0.03      0.00      0.01       274
           1       0.95      0.99      0.97      4910

    accuracy                           0.94      5184
   macro avg       0.49      0.50      0.49      5184
weighted avg       0.90      0.94      0.92      5184



In [14]:
y_pred=rf.predict(X_test)

In [15]:
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, y_pred)
print('Test Accuracy:', accuracy)
print('Classification Report:')
print(classification_report(y_test, y_pred))

Test Accuracy: 0.9443656422379827
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2820
           1       0.94      1.00      0.97     47940

    accuracy                           0.94     50760
   macro avg       0.47      0.50      0.49     50760
weighted avg       0.89      0.94      0.92     50760



Test accuracy is better than Validation accuracy

In [16]:
y_train.value_counts()

binary_fault
1    19570
0     1166
Name: count, dtype: int64

We should know that the data we are working on is quite imbalanced. Therefore, we can use class weight, oversampling and undersampling methods to have a better and more robust model.

In [17]:
rf_1=RandomForestClassifier(class_weight='balanced', random_state=42)
rf_1.fit(X_train, y_train)
y_pred = rf_1.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.22      0.06      0.10      2820
           1       0.95      0.99      0.97     47940

    accuracy                           0.94     50760
   macro avg       0.58      0.52      0.53     50760
weighted avg       0.91      0.94      0.92     50760



In [18]:
accuracy = accuracy_score(y_test, y_pred)
print('Test Accuracy:', accuracy)

Test Accuracy: 0.9355397951142632


In [19]:
y_train.shape

(20736,)

In [20]:
from imblearn.over_sampling import RandomOverSampler
RO=RandomOverSampler(random_state=42)
X_train_ro, y_train_ro=  RO.fit_resample(X_train,y_train)

In [21]:
rf.fit(X_train_ro,y_train_ro)

RandomForestClassifier()

In [22]:
y_pred=rf.predict(X_test)

In [23]:
accuracy = accuracy_score(y_test, y_pred)
print('Test Accuracy:', accuracy)
print('Classification Report:')
print(classification_report(y_test, y_pred))

Test Accuracy: 0.9191095350669819
Classification Report:
              precision    recall  f1-score   support

           0       0.22      0.18      0.20      2820
           1       0.95      0.96      0.96     47940

    accuracy                           0.92     50760
   macro avg       0.59      0.57      0.58     50760
weighted avg       0.91      0.92      0.92     50760



In [24]:
from imblearn.under_sampling import RandomUnderSampler
RU=RandomUnderSampler(random_state=42)
X_train_ru,y_train_ru=RU.fit_resample(X_train,y_train)

In [25]:
rf_2= RandomForestClassifier(random_state=42)
rf_2.fit(X_train_ru,y_train_ru)

RandomForestClassifier(random_state=42)

In [26]:
y_pred_ru=rf_2.predict(X_test)

In [27]:
accuracy = accuracy_score(y_test, y_pred_ru)
print('Test Accuracy:', accuracy)
print('Classification Report:')
print(classification_report(y_test, y_pred_ru))

Test Accuracy: 0.7093183609141056
Classification Report:
              precision    recall  f1-score   support

           0       0.16      0.95      0.27      2820
           1       1.00      0.70      0.82     47940

    accuracy                           0.71     50760
   macro avg       0.58      0.82      0.54     50760
weighted avg       0.95      0.71      0.79     50760



Here we may observe that although the accuracy is less, we are having a better recall and f1-score for both 0 and 1 which are better parameters to understand an imbalance dataset

# Now let us proceed with MultiClass classification

In [28]:
X_train = df_train.drop(columns=drop_cols, errors='ignore')
y_train = df_train['faultNumber']

In [29]:
y_train.value_counts()

faultNumber
0.0     1440
1.0     1440
19.0    1440
18.0    1440
17.0    1440
16.0    1440
14.0    1440
13.0    1440
12.0    1440
11.0    1440
10.0    1440
8.0     1440
7.0     1440
6.0     1440
5.0     1440
4.0     1440
2.0     1440
20.0    1440
Name: count, dtype: int64

Now, this a balanced dataset where each class has almost same number of points

In [30]:
X_test = df_test.drop(columns=drop_cols, errors='ignore')
y_test = df_test['faultNumber']

In [31]:
y_test.value_counts()

faultNumber
0     2820
1     2820
19    2820
18    2820
17    2820
16    2820
14    2820
13    2820
12    2820
11    2820
10    2820
8     2820
7     2820
6     2820
5     2820
4     2820
2     2820
20    2820
Name: count, dtype: int64

Even the test dataset is balanced

In [32]:
common_cols = sorted(list(set(X_train.columns).intersection(set(X_test.columns))))

In [33]:
X_train = X_train[common_cols]
X_test  = X_test[common_cols]

In [34]:
rf=RandomForestClassifier(n_estimators=100,random_state=42)

In [35]:
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [36]:
y_pred = rf.predict(X_test)

In [37]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.67


In [38]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.19      0.55      0.28      2820
           1       0.96      0.84      0.90      2820
           2       1.00      0.84      0.91      2820
           4       0.93      0.83      0.88      2820
           5       0.27      0.29      0.28      2820
           6       1.00      0.85      0.92      2820
           7       1.00      0.85      0.92      2820
           8       0.91      0.61      0.73      2820
          10       0.65      0.45      0.53      2820
          11       0.82      0.66      0.73      2820
          12       0.66      0.47      0.55      2820
          13       0.97      0.68      0.80      2820
          14       0.98      0.82      0.89      2820
          16       0.29      0.40      0.33      2820
          17       0.91      0.78      0.84      2820
          18       0.69      0.76      0.72      2820
          19       0.59      0.71      0.65      2820
   

In [39]:
import numpy as np

# Ensure exactly 37 features (removed one value)
new_data = np.array([[12.5, 4.3, 18.6, 22.1, 7.9, 5.5, 16.2, 3.8, 11.9, 14.0,
                      20.3, 9.8, 13.1, 17.5, 15.6, 19.2, 10.4, 6.7, 8.9, 12.3,
                      11.5, 14.8, 16.9, 7.3, 9.5, 15.0, 18.2, 19.8, 13.9, 10.7,
                      16.1, 12.2, 7.8, 14.4, 11.0, 13.7, 17.3]])  # Now 37 values


# Predict
fault_prediction = rf.predict(new_data)

# Result
if fault_prediction[0] == 0:
    print("The data point is Non-Faulty ")
else:
    print(f"The data point is Faulty  with Fault Number: {fault_prediction[0]}")

The data point is Faulty  with Fault Number: 18.0


C:\Users\ADITI\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
